# MobileViT-XS PyTorch Model - Quantization for IMX500

[Run this tutorial in Google Colab](https://colab.research.google.com/github/sony/model_optimization/blob/main/tutorials/notebooks/imx500_notebooks/pytorch/pytorch_mobilevit_xs_for_imx500.ipynb)

## Overview

In this tutorial, we will illustrate a basic and quick process of preparing a pre-trained model for deployment using MCT. Specifically, we will demonstrate how to download a pre-trained MobileViT-XS model from the MCT Models Library, compress it, and make it deployment-ready using MCT's post-training quantization techniques.

We will use an existing pre-trained MobileViT-XS model based on [Timm](https://github.com/huggingface/pytorch-image-models). The model was slightly adjusted for model quantization, particularly aimed at preventing folding along the batch axis. We will quantize the model using MCT post training quantization and evaluate the performance of the floating point model and the quantized model on ImageNet dataset.


## Summary

In this tutorial we will cover:

1. Post-Training Quantization using MCT of PyTorch classification model.
2. Data preparation - loading and preprocessing validation and representative datasets from ImageNet.
3. Accuracy evaluation of the floating-point and the quantized models.

## Setup
### Install the relevant packages

In [ ]:
!pip install -q torch
!pip install onnx
!pip install timm
!pip install 'huggingface-hub>=0.21.0'

Install MCT (if it’s not already installed). Additionally, in order to use all the necessary utility functions for this tutorial, we also copy [MCT tutorials folder](https://github.com/sony/model_optimization/tree/main/tutorials) and add it to the system path.

In [ ]:
import sys
import importlib

if not importlib.util.find_spec('model_compression_toolkit'):
    !pip install model_compression_toolkit
!git clone https://github.com/sony/model_optimization.git temp_mct && mv temp_mct/tutorials . && \rm -rf temp_mct
sys.path.insert(0,"tutorials")


### Download ImageNet validation set
Download ImageNet dataset with only the validation split.

Note that for demonstration purposes we use the validation set for the model quantization routines. Usually, a subset of the training dataset is used, but loading it is a heavy procedure that is unnecessary for the sake of this demonstration.

This step may take several minutes...

In [ ]:
import os
if not os.path.isdir('imagenet'):
    !mkdir imagenet
    !wget https://image-net.org/data/ILSVRC/2012/ILSVRC2012_devkit_t12.tar.gz
    !mv ILSVRC2012_devkit_t12.tar.gz imagenet/
    !wget https://image-net.org/data/ILSVRC/2012/ILSVRC2012_img_val.tar
    !mv ILSVRC2012_img_val.tar imagenet/

## Model Quantization

### Download a Pre-Trained Model 

We begin by loading a pre-trained [MobileViT-XS](https://huggingface.co/SSI-DNN/pytorch_mobilevit_xs) model. This implementation is based on [Timm](https://github.com/huggingface/pytorch-image-models) and includes a slightly modified version of timm/Attention module that was adapted for model quantization, particularly aimed at preventing folding along the batch axis. For further insights into the model's implementation details, please refer to [MCT Models Garden - yolov8](https://github.com/sony/model_optimization/tree/main/tutorials/mct_model_garden/models_pytorch/mobilevit_xs).  

In [ ]:
from tutorials.mct_model_garden.models_pytorch.mobilevit_xs.mobilevit_xs import MobileViTXSPyTorch
model = MobileViTXSPyTorch.from_pretrained("SSI-DNN/pytorch_mobilevit_xs")

### Post training quantization using Model Compression Toolkit 

Now, we're all set to use MCT's post-training quantization. To begin, we'll define a representative dataset and proceed with the model quantization. Please note that, for demonstration purposes, we'll use the evaluation dataset as our representative dataset. We'll calibrate the model using 80 representative images, divided into 20 iterations of 'batch_size' images each. 

Additionally, to further compress the model's memory footprint, we will employ the mixed-precision quantization technique. This method allows each layer to be quantized with different precision options: 2, 4, and 8 bits, aligning with the imx500 target platform capabilities.

In [ ]:
import model_compression_toolkit as mct
from model_compression_toolkit.core.pytorch.pytorch_device_config import get_working_device
from timm.data import create_loader, resolve_data_config
from typing import Iterator, Tuple, List
import torchvision


BATCH_SIZE = 4
n_iters = 20
IMG_SIZE = 256
DATA_ARGS = {'img_size': IMG_SIZE}
device = get_working_device()

# Load representative dataset
data_config = resolve_data_config(args=DATA_ARGS,
                                  model=model)

# Extract ImageNet validation dataset using torchvision "datasets" module
val_dataset = torchvision.datasets.ImageNet(root='./imagenet', split='val')
    
representative_dataset = create_loader(
    val_dataset,
    input_size=data_config['input_size'],
    batch_size=BATCH_SIZE,
    interpolation=data_config['interpolation'],
    mean=data_config['mean'],
    std=data_config['std'],
    crop_pct=data_config['crop_pct'],
    device=device)

# Define representative dataset generator
def get_representative_dataset(n_iter: int, dataset_loader: Iterator[Tuple]):
    """
    This function creates a representative dataset generator. The generator yields numpy
        arrays of batches of shape: [Batch, H, W ,C].
    Args:
        n_iter: number of iterations for MCT to calibrate on
    Returns:
        A representative dataset generator
    """       
    def representative_dataset() -> Iterator[List]:
        ds_iter = iter(dataset_loader)
        for _ in range(n_iter):
            yield [next(ds_iter)[0]]

    return representative_dataset

# Get representative dataset generator
representative_dataset_gen = get_representative_dataset(n_iter=n_iters,
                                                        dataset_loader=representative_dataset)

# Set IMX500-v1 TPC
tpc = mct.get_target_platform_capabilities(fw_name="pytorch",
                                           target_platform_name='imx500',
                                           target_platform_version='v1')

# We adjusted the quantization configuration to match the model, activated the shift negative activation correction, and set the z-threshold to 7.0. This is a general best practice for transformer architectures.
ptq_config = mct.core.CoreConfig(quantization_config=mct.core.QuantizationConfig(
    shift_negative_activation_correction=True,
    z_threshold=7.0))

# Perform post training quantization
quant_model, _ = mct.ptq.pytorch_post_training_quantization(in_module=model,
                                                            representative_data_gen=representative_dataset_gen,
                                                            core_config=ptq_config,
                                                            target_platform_capabilities=tpc)
print('Quantized model is ready')

### Model Export

Now, we can export the quantized model, ready for deployment, into a `.onnx` format file. Please ensure that the `save_model_path` has been set correctly. 

In [ ]:
mct.exporter.pytorch_export_model(model=quant_model,
                                  save_model_path='./qmodel.onnx',
                                  repr_dataset=representative_dataset_gen, 
                                  onnx_opset_version=17)

## Evaluation on ImageNet dataset

### Floating point model evaluation
Please ensure that the dataset path has been set correctly before running this code cell.

In [ ]:
from tutorials.resources.utils.pytorch_tutorial_tools import classification_eval

val_loader = create_loader(
    val_dataset,
    input_size=data_config['input_size'],
    batch_size=BATCH_SIZE,
    interpolation=data_config['interpolation'],
    mean=data_config['mean'],
    std=data_config['std'],
    crop_pct=data_config['crop_pct'],
    device=device)

# Evaluate the model on ImageNet
eval_results = classification_eval(model, val_loader)

# Print float model Accuracy results
print("Float model Accuracy: {:.4f}".format(round(100 * eval_results[0], 2)))

### Quantized model evaluation
We can evaluate the performance of the quantized model. There is a slight decrease in performance that can be further mitigated by either expanding the representative dataset or employing MCT's advanced quantization methods, such as GPTQ (Gradient-Based/Enhanced Post Training Quantization).

In [ ]:
# Evaluate the quantized model on ImageNet
eval_results = classification_eval(quant_model, val_loader)

# Print quantized model Accuracy results
print("Quantized model Accuracy: {:.4f}".format(round(100 * eval_results[0], 2)))

\
Copyright 2024 Sony Semiconductor Israel, Inc. All rights reserved.

Licensed under the Apache License, Version 2.0 (the "License");
you may not use this file except in compliance with the License.
You may obtain a copy of the License at

    http://www.apache.org/licenses/LICENSE-2.0

Unless required by applicable law or agreed to in writing, software
distributed under the License is distributed on an "AS IS" BASIS,
WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
See the License for the specific language governing permissions and
limitations under the License.